<a href="https://colab.research.google.com/github/Nivetha1316/Nivetha/blob/main/flight_delay_prediction%20task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import imblearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score


In [ ]:
dataset=pd.read_csv("/content/flight_data_2018_to_2022.csv")
dataset.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12872 entries, 0 to 12871
Columns: 120 entries, Year to Unnamed: 119
dtypes: float64(76), int64(23), object(21)
memory usage: 11.8+ MB


In [ ]:
dataset=dataset.drop('Unnamed: 119', axis=1)
dataset.isnull().sum()

Year                    0
Quarter                 0
Month                   0
DayofMonth              0
DayOfWeek               0
                    ...  
Div5TotalGTime      12872
Div5LongestGTime    12872
Div5WheelsOff       12872
Div5TailNum         12872
Duplicate               1
Length: 119, dtype: int64

In [ ]:
dataset=dataset[["Month","DayofMonth","Origin","Dest","CRSArrTime","DepDelay","ArrDelay"]]
dataset.isnull().sum()

Month           0
DayofMonth      0
Origin          0
Dest            0
CRSArrTime      0
DepDelay      783
ArrDelay      804
dtype: int64

In [ ]:
dataset[dataset.isnull().any(axis=1)].head(10)

,Month,DayofMonth,Origin,Dest,CRSArrTime,DepDelay,ArrDelay
0,1,6,FLL,LGA,1419,NaN,NaN
30,1,6,DTW,BNA,2216,NaN,NaN
53,1,6,ATL,ORF,2012,-2.0,NaN
75,1,6,CVG,BOS,1903,NaN,NaN
106,1,6,IAD,SEA,2019,-1.0,NaN
148,1,6,MIA,LGA,2307,NaN,NaN
157,1,6,ATL,BNA,1646,NaN,NaN
158,1,6,BNA,ATL,2010,NaN,NaN
201,1,6,EWR,ATL,829,NaN,NaN
242,1,6,TPA,LGA,2053,NaN,NaN


In [ ]:
dataset['DepDelay'].mode()

0   -3.0
Name: DepDelay, dtype: float64

In [ ]:
dataset=dataset.fillna({'ArrDelay':1})
dataset=dataset.fillna({'DepDelay':0})
dataset.iloc[177:185]

,Month,DayofMonth,Origin,Dest,CRSArrTime,DepDelay,ArrDelay
177,1,6,CHS,ATL,1148,-8.0,-23.0
178,1,6,AUS,DTW,930,-9.0,-21.0
179,1,6,TPA,LGA,1743,-5.0,-24.0
180,1,6,SJU,ATL,1110,-6.0,-24.0
181,1,6,ATL,STX,1440,86.0,77.0
182,1,6,STX,ATL,1913,61.0,36.0
183,1,6,STT,ATL,1747,-7.0,-27.0
184,1,6,SJU,JFK,2159,-8.0,-40.0


In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['Dest'] = le.fit_transform(dataset['Dest'])
dataset['Origin'] = le.fit_transform(dataset['Origin'])

In [21]:
dataset.head(5)

,Month,DayofMonth,Origin,Dest,CRSArrTime,DepDelay,ArrDelay
0,1,6,40,66,1419,0.0,1.0
1,1,6,3,40,1821,-4.0,-1.0
2,1,6,40,3,2127,-2.0,-12.0
3,1,6,40,100,1227,-5.0,-15.0
4,1,6,3,59,1142,-4.0,-11.0


In [22]:
dataset['Origin'].unique()

array([ 40,   3,  59, 101,  80, 100,   9,  57,  66, 108, 129, 107, 124,
       115,   7,  60,  11, 105, 110,  72,  75,  31,  95,  71,  21,  17,
        35,  14,  93,  23,  61,  63,  92,  27,  90,  41,  64,  29,  74,
       114, 104,  89, 109,  22,  96,  94,  54,  19,  26,  99,  58, 117,
        70,  67,  10,  81, 125, 120,  46,  78,   5, 121,  84,  28,  15,
        69,  51,  32,  24, 118, 106, 128,  53,  55, 113,  91, 119,  83,
        20,  73,  88,  87, 111,  76,   2,  37,  52,   8, 127,  86,  49,
        18,  16,  30,  36,  56,  97, 103,  47, 123,  42,   4,  43,  39,
        77,  48,  65,  12,   6, 102,  44, 112,  25,  98, 116,   1,  79,
       122,  33,  50,  68,  85,  62,   0, 126,  34,  13,  38,  45,  82])

In [23]:
dataset = pd.get_dummies(dataset,columns=['Origin','Dest'])
dataset.head()

,Month,DayofMonth,CRSArrTime,DepDelay,ArrDelay,Origin_0,Origin_1,Origin_2,Origin_3,Origin_4,...,Dest_120,Dest_121,Dest_122,Dest_123,Dest_124,Dest_125,Dest_126,Dest_127,Dest_128,Dest_129
0,1,6,1419,0.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,6,1821,-4.0,-1.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,2127,-2.0,-12.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,1227,-5.0,-15.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,6,1142,-4.0,-11.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
x = dataset.iloc[:,0:8].values
y = dataset.iloc[:,8:9].values

In [25]:
x

array([[1.000e+00, 6.000e+00, 1.419e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 6.000e+00, 1.821e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 6.000e+00, 2.127e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.000e+00, 2.900e+01, 7.440e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 2.900e+01, 1.041e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 2.900e+01, 2.212e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [26]:
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder()
z=oh.fit_transform(x[:,4:5]).toarray()
t=oh.fit_transform(x[:,5:6]).toarray()


In [27]:
z

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
t

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [29]:
x=np.delete(x,[4,5],axis=1)